In [1]:
using Pkg; Pkg.activate(@__DIR__)

using Revise
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging
using Optim
using ModelingToolkit
using OrdinaryDiffEq
using StaticArrays
using DataStructures
using LinearAlgebra
using TimerOutputs
# using Makie, GLMakie

  Activating project at `~/globtim/experiments/week8`


In [7]:
Revise.includet(joinpath(@__DIR__, "../../Examples/systems/DynamicalSystems.jl"))
using .DynamicalSystems

In [8]:
model, params, states, outputs = define_simple_1D_model_locally_identifiable()

(Model model:
Equations (1):
  1 standard: see equations(model)
Unknowns (1): see unknowns(model)
  x1(t)
Parameters (1): see parameters(model)
  a, Num[a], Num[x1(t)], Equation[y1(t) ~ x1(t)])

In [9]:
model

Model model:
Equations (1):
  1 standard: see equations(model)
Unknowns (1): see unknowns(model)
  x1(t)
Parameters (1): see parameters(model)
  a

In [ ]:
n = 1
ic = [0.3]
p_true = [0.1]
num_points = 20
time_interval = [0.0, 1.0]

2-element Vector{Float64}:
 0.0
 1.0

$$\operatorname{error}(y, \hat{y}) = \sum_{t_i}(y(t_i) - \hat{y}(t_i))^2$$

In [14]:
error_func = make_error_distance(
    model,
    outputs,
    ic,
    p_true,
    time_interval,
    num_points,
    L2_norm,
)

(::Main.DynamicalSystems.var"#Error_distance#9"{Vector{Equation}, Vector{Float64}, Int64, Main.DynamicalSystems.var"#Error_distance#5#10"{System, Vector{Float64}, typeof(L2_norm), typeof(first), OrderedDict{Any, Vector{Float64}}, ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, MTKParameters{Vector{Float64}, Vector{Float64}, Tuple{}, Tuple{}, Tuple{}, Tuple{}}, ODEFunction{true, SciMLBase.FullSpecialize, ModelingToolkit.GeneratedFunctionWrapper{(2, 3, true), RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:__mtk_arg_1, :___mtkparameters___, :t), ModelingToolkit.var"#_RGF_ModTag", ModelingToolkit.var"#_RGF_ModTag", (0x16bb48ba, 0xd6fa5e9e, 0x19926d18, 0x5fa42af9, 0x8fc46a1f), Nothing}, RuntimeGeneratedFunctions.RuntimeGeneratedFunction{(:ˍ₋out, :__mtk_arg_1, :___mtkparameters___, :t), ModelingToolkit.var"#_RGF_ModTag", ModelingToolkit.var"#_RGF_ModTag", (0x86f82891, 0x8d9e62ea, 0x469bab5f, 0x2e3ea9ea, 0x3ba10142), Nothing}}, UniformScaling{Bool}, Nothing, Nothing, Nothing,

In [15]:
error_func([0.05]), error_func([0.1]), error_func([0.11])

(0.029923526027846307, 0.0, 0.008378587287810099)

In [20]:
p_center = [p_true[1][1] + 0.05]
GN = 30
sample_range = 0.4
d = (:one_d_for_all, 5)
basis = :chebyshev
precision = Globtim.RationalPrecision

RationalPrecision::PrecisionType = 1

In [21]:
@polyvar(x[1:n]); # Define polynomial ring
TR = test_input(
    error_func,
    dim = n,
    center = p_center,
    GN = GN,
    sample_range = sample_range,
);

pol_cheb = Constructor(
    TR,
    d,
    basis = basis,
    precision = precision,
    verbose = true,
)
real_pts_cheb, (wd_in_std_basis, _sys, _nsols) = solve_polynomial_system(
    x,
    n,
    d,
    pol_cheb.coeffs;
    basis = basis,
    return_system = true,
)
df_cheb = process_crit_pts(real_pts_cheb, error_func, TR)

@info "" df_cheb


Condition number of G: 2.0000000000000018
Chosen method: LinearSolve.LUFactorization{RowMaximum}
current L2-norm: 0.018462604930679367
Tracking 4 paths...  75%|███████████████████████▎       |  ETA: 0:00:03



Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:09
                   # paths tracked: 4
   # non-singular solutions (real): 4 (2)
       # singular endpoints (real): 0 (0)
          # total solutions (real): 4 (2)


┌ Info: 
│   df_cheb = 1×2 DataFrame
 Row │ x1           z         
     │ Float64      Float64   
─────┼────────────────────────
   1 │ -0.00962447  0.0395285
└ @ Main /home/demin/globtim/experiments/week8/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W5sdnNjb2RlLXJlbW90ZQ==.jl:27


In [22]:
wd_in_std_basis

-0.031187522586432537 + 0.2816550341755207x₁ + 0.48989000015058415x₁² + 0.2761879149666276x₁³ - 0.000796453032651728x₁⁴ - 0.17991723723924818x₁⁵

In [24]:
_sys

System of length 1
 1 variables: x₁

 0.281655034175521 + 0.979780000301168*x₁ + 0.828563744899883*x₁^2 - 0.00318581213060691*x₁^3 - 0.899586186196241*x₁^4

In [ ]:
const T = Float64

using DynamicPolynomials
using HomotopyContinuation, ProgressLogging

config = (
    n = 1,
    d = (:one_d_for_all, 20),
    GN = 300,
    time_interval = T[0.0, 1.0],
    p_true = [T[0.1], T[-0.1]],
    ic = T[0.3],
    num_points = 20,
    sample_range = 0.4,
    distance = L2_norm,
    model_func = define_simple_1D_model_locally_identifiable,
    basis = :chebyshev,
    precision = Globtim.RationalPrecision,
    my_eps = 0.02,
    fine_step = 0.002,
)
config = merge(
    config,
    (;
        plot_range = [
            -(config.sample_range+config.my_eps):config.fine_step:(config.sample_range+config.my_eps),
        ],
        p_center = [config.p_true[1][1] + 0.05],
    ),
)

model, params, states, outputs = config.model_func()

error_func = make_error_distance(
    model,
    outputs,
    config.ic,
    config.p_true[1],
    config.time_interval,
    config.num_points,
    config.distance,
)

@polyvar(x[1:config.n]); # Define polynomial ring
TR = test_input(
    error_func,
    dim = config.n,
    center = config.p_center,
    GN = config.GN,
    sample_range = config.sample_range,
);

pol_cheb = Constructor(
    TR,
    config.d,
    basis = config.basis,
    precision = config.precision,
    verbose = true,
)
real_pts_cheb, (wd_in_std_basis, _sys, _nsols) = solve_polynomial_system(
    x,
    config.n,
    config.d,
    pol_cheb.coeffs;
    basis = pol_cheb.basis,
    return_system = true,
)
df_cheb = process_crit_pts(real_pts_cheb, error_func, TR)

@info "" df_cheb

id = "id_1D"
filename = "$(id)_$(config.model_func)_$(config.distance)"

open(joinpath(@__DIR__, "images", "$filename.txt"), "w") do io
    println(io, "config = ", config, "\n\n")
    println(io, "Condition number of the Vandermonde system: ", pol_cheb.cond_vandermonde)
    println(io, "L2 norm (error of approximation): ", pol_cheb.nrm)
    println(io, "Polynomial system:")
    println(io, "   Number of sols: ", _nsols)
    println(
        io,
        "   Bezout bound: ",
        map(eq -> HomotopyContinuation.ModelKit.degree(eq), _sys),
        " which is ",
        prod(map(eq -> HomotopyContinuation.ModelKit.degree(eq), _sys)),
    )
    println(io, "Critical points found:\n", df_cheb)
    if !isempty(df_cheb)
        println(io, "Number of critical points: ", nrow(df_cheb))
    else
        println(io, "No critical points found.")
    end
    println(io, _TO)
end

println(_TO)

problem = ODEProblem(
    ModelingToolkit.complete(model),
    merge(
        Dict(ModelingToolkit.unknowns(model) .=> config.ic),
        Dict(ModelingToolkit.parameters(model) .=> config.p_true[1]),
    ),
    config.time_interval,
)
data_sample_true = sample_data(
    problem,
    model,
    outputs,
    config.time_interval,
    config.p_true[1],
    config.ic,
    config.num_points,
)
Y_true = data_sample_true[first(keys(data_sample_true))]
println("Sampled data (true): ", Y_true)

if true
    fig = Globtim.plot_error_function_1D_with_critical_points(
        pol_cheb,
        TR,
        df_cheb,
        x,
        wd_in_std_basis,
        config.p_true,
        config.plot_range,
        config.distance;
        xlabel = "Parameter 1",
        model_func = config.model_func,
    )

    display(fig)

    # Makie.save(
    #     joinpath(@__DIR__, "images", "$filename.png"),
    #     fig,
    #     px_per_unit = 1.5,
    # )
end
